### APIs

In [ ]:
# pip install requests
!pip install requests
!pip install python-dotenv

### HTTP Methods (CRUD Operations)

In [ ]:

# GET = SELECT (read data)
# Used to retrieve/read data from a server
# Example: Get list of employees

# POST = INSERT (create new data)
# Used to create new records
# Example: Create a new employee record

# PUT = UPDATE (modify existing data)
# Used to update/replace existing records
# Example: Update employee salary

# DELETE = DELETE (remove data)
# Used to remove records
# Example: Delete an employee record

### Status Codes

In [ ]:
# Status codes tell you if your request worked or not
# Like SY-SUBRC in ABAP!

# 2xx = Success (like SY-SUBRC = 0)
200  # OK - Request successful
201  # Created - New record created successfully

# 4xx = Client Error (like SY-SUBRC = 4)
400  # Bad Request - Your request has errors
401  # Unauthorized - You need to login/authenticate
404  # Not Found - The resource doesn't exist

# 5xx = Server Error (like SY-SUBRC = 8)
500  # Internal Server Error - Something broke on the server

### Requests Library

In [ ]:
import requests
import uuid
from dotenv import load_dotenv
import os
import json

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

url = "https://apim-test.jgsummit.com.ph:8443/qas/api/employee-profile/v2/employee-details" #qas
correlation_id = str(uuid.uuid4())

headers = {        
        "client-id": client_id,
        "client-secret": client_secret,
        "x-correlation-id": correlation_id
        
    }

    # Data payload for the GET request
data = {
        "field": "email-address",        
        "value": "STEPHEN.BONIFACIO@JGSUMMIT.PH",   
        "integrationType": "outbound",
        "systemId": "JHQ",
        "program": "MIGO_PERSONAL_INFO",      
        "dateFrom": "20250101",        
        "dateTo": "20251231"
    }

response = requests.get(url, params=data, headers=headers)
response_data = response.json()
print(json.dumps(response_data, indent=4))
print(response.reason)
print(response.status_code)

{
    "responseCode": "200",
    "sapSystemId": "JHQ",
    "resource": "http://localhost:8905/api/urc-hcm/s/v1/employee-details/(perner)",
    "transactionId": "5c537e79-cf29-4aab-814b-b72ff3452d18",
    "timestamp": "2025-10-22 04:00:21",
    "details": {
        "status": "Success",
        "statusMessage": "Retrieved employee data from SAP HCM successfully"
    },
    "data": [
        {
            "tOutput": {
                "Permanentresidence_StreetandHouseNo.": "6 Ortona, Athena Classique Phase 1",
                "Permanentresidence_2ndAddressLine": "Brgy Anabu II-A, Imus Cavite",
                "Permanentresidence_City": "IMUS, CAVITE",
                "Permanentresidence_PostalCode/City": "4103",
                "Currentresidence_StreetandHouseNo.": "125 Brgy Buting",
                "Currentresidence_City": "CABUYAO, LAGUNA",
                "Currentresidence_PostalCode/City": "1600",
                "TaxIdentificationNumber": "000190863787",
                "SocialSecuri

### Working with Azure Storage

In [1]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from io import StringIO

load_dotenv()

url = "https://synapsedl50415.blob.core.windows.net/cit-training/employees.csv"
sas_token = os.getenv("SAS_TOKEN")

full_url = url + "?" + sas_token

response = requests.get(full_url)
    
df = pd.read_csv(StringIO(response.text))
df.head()

,EmployeeID,FirstName,LastName,Department,Position,City,Age,Salary,DateHired
0,E1000,Emma,Gutierrez,IT,Executive,Makati,58,86907,2020-07-08
1,E1001,Maria,Perez,HR,Executive,Taguig,33,88777,2019-10-14
2,E1002,Diego,Santos,Operations,Clerk,Manila,24,43977,2017-12-14
3,E1003,Isabel,Rivera,Finance,Engineer,Taguig,22,73624,2019-04-19
4,E1004,John,Hernandez,Finance,Executive,Makati,54,93321,2014-07-25


### Working with NoSQL (Azure CosmosDB)

In [ ]:
import requests
import hmac
import hashlib
import base64
from datetime import datetime
from urllib.parse import quote
import os
from dotenv import load_dotenv

load_dotenv()

# Configuration
COSMOS_ENDPOINT = "https://jgs-migo-cdb-dev.documents.azure.com:443/"
COSMOSDB_PRIMARY_KEY = os.getenv("COSMOSDB_PRIMARY_KEY")
DATABASE_ID = "migo-chat-history-dev"
CONTAINER_ID = "chat-history"

# util function to generate auth token
def generate_auth_token(verb, resource_type, resource_id, date):
    """Generate authorization token for Cosmos DB"""
    key = base64.b64decode(COSMOSDB_PRIMARY_KEY)
    text = f"{verb.lower()}\n{resource_type.lower()}\n{resource_id}\n{date.lower()}\n\n"
    
    signature = base64.b64encode(
        hmac.new(key, text.encode('utf-8'), hashlib.sha256).digest()
    ).decode()
    
    return quote(f"type=master&ver=1.0&sig={signature}")

In [ ]:
query = "SELECT * FROM c WHERE c.userId = 'Stephen.Bonifacio@jgsummit.ph'"

resource_link = f"dbs/{DATABASE_ID}/colls/{CONTAINER_ID}"
date = datetime.utcnow().strftime('%a, %d %b %Y %H:%M:%S GMT')

auth_token = generate_auth_token("POST", "docs", resource_link, date)

headers = {
    "Authorization": auth_token,
    "x-ms-date": date,
    "x-ms-version": "2018-12-31",
    "x-ms-documentdb-isquery": "True",
    "x-ms-documentdb-query-enablecrosspartition": "True",
    "Content-Type": "application/query+json"
}

body = {
    "query": query
}

url = f"{COSMOS_ENDPOINT}/{resource_link}/docs"
response = requests.post(url, headers=headers, json=body)

json_res = response.json()

json_res.get('Documents')[0]


{'id': 'a609422e-2443-4d15-93ce-1078cc1fe3ea',
 'app_id': '38475730',
 'userId': 'Stephen.Bonifacio@jgsummit.ph',
 'type': 'conversation',
 'title': 'hi',
 'prompt_tokens': 95753,
 'completion_tokens': 5747,
 'total_tokens': 101500,
 'category': None,
 'createdAt': '2025-05-27T07:07:21Z',
 'updatedAt': '2025-06-05T05:47:27Z',
 '_rid': 'OrxkAIfzDXlwuwAAAAAAAA==',
 '_self': 'dbs/OrxkAA==/colls/OrxkAIfzDXk=/docs/OrxkAIfzDXlwuwAAAAAAAA==/',
 '_etag': '"2e00be1b-0000-1800-0000-68412f760000"',
 '_attachments': 'attachments/',
 '_ts': 1749102454}

### ETL: extract (azure storage) > transform (python) > load (azure sql)